In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [908 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:14 http://ppa.

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/u/0/uc?id=1kCXnIeoPT6p9kS_ANJ0mmpxlfDwK1yio&export=download
!unzip DatasetToCompleteTheSixSparkExercises.zip

In [38]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.config('spark.ui.port', '4050')\
                            .getOrCreate()

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2023-03-12 21:39:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  3.38MB/s    in 5.1s    

2023-03-12 21:40:00 (2.60 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [5]:
!./ngrok authtoken #Seu token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
get_ipython().system_raw('./ngrok http 4050 &')
#Aguarde 10s

In [7]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://081c-35-199-145-12.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://081c-35-199-145-12.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [39]:
sales_dtf = spark.read.parquet("sales_parquet")
products_dtf = spark.read.parquet("products_parquet")
sellers_dtf = spark.read.parquet("sellers_parquet")

sales_dtf.show(5)
sellers_dtf.show(5)
products_dtf.show(5)

+--------+----------+---------+----------+---------------+--------------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+--------+----------+---------+----------+---------------+--------------------+
|       1|         0|        0|2020-07-10|             26|kyeibuumwlyhuwksx...|
|       2|         0|        0|2020-07-08|             13|jfyuoyfkeyqkckwbu...|
|       3|         0|        0|2020-07-05|             38|uyjihlzhzcswxcccx...|
|       4|         0|        0|2020-07-05|             56|umnxvoqbdzpbwjqmz...|
|       5|         0|        0|2020-07-05|             11|zmqexmaawmvdpqhih...|
+--------+----------+---------+----------+---------------+--------------------+
only showing top 5 rows

+---------+-----------+------------+
|seller_id|seller_name|daily_target|
+---------+-----------+------------+
|        0|   seller_0|     2500000|
|        1|   seller_1|      257237|
|        2|   seller_2|      754188|
|        3|   seller_3|      3104

In [40]:
sales_dtf = sales_dtf.cache()
sales_dtf.count()

20000040

In [41]:
sales_dtf.count()

20000040

In [22]:
sales_dtf.rdd.getNumPartitions()

83

In [23]:
joined = sales_dtf.join(products_dtf,on="product_id")

In [24]:
joined.show()

+----------+--------+---------+----------+---------------+--------------------+----------------+-----+
|product_id|order_id|seller_id|      date|num_pieces_sold|       bill_raw_text|    product_name|price|
+----------+--------+---------+----------+---------------+--------------------+----------------+-----+
|  10005243|12478308|        6|2020-07-04|             98|qfvpgiscflyjxphcq...|product_10005243|   44|
|  10023464| 8996776|        9|2020-07-03|             59|jjbyqkzcimBfoehbv...|product_10023464|   19|
|  10050363|10476976|        6|2020-07-03|             18|xqhlvkpxtzrfdadry...|product_10050363|   98|
|  10089524| 5977582|        2|2020-07-01|             53|jchvhzbeaicqitpvx...|product_10089524|  100|
|  10122266| 1482892|        2|2020-07-07|             25|kezhpglnqigaqwrss...|product_10122266|   89|
|  10134574| 2987749|        3|2020-07-03|             28|bmtlxvobfmerqlckf...|product_10134574|  138|
|  10150439|15982399|        4|2020-07-01|             72|rvgiffufcxceotp

In [42]:
sales_dtf.explain()

== Physical Plan ==
InMemoryTableScan [order_id#1496, product_id#1497, seller_id#1498, date#1499, num_pieces_sold#1500, bill_raw_text#1501]
   +- InMemoryRelation [order_id#1496, product_id#1497, seller_id#1498, date#1499, num_pieces_sold#1500, bill_raw_text#1501], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) ColumnarToRow
            +- FileScan parquet [order_id#1496,product_id#1497,seller_id#1498,date#1499,num_pieces_sold#1500,bill_raw_text#1501] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/content/sales_parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<order_id:string,product_id:string,seller_id:string,date:string,num_pieces_sold:string,bill...




In [45]:
sales_dtf.groupby("seller_id").count().explain()

== Physical Plan ==
*(2) HashAggregate(keys=[seller_id#1498], functions=[count(1)])
+- Exchange hashpartitioning(seller_id#1498, 200), true, [id=#479]
   +- *(1) HashAggregate(keys=[seller_id#1498], functions=[partial_count(1)])
      +- InMemoryTableScan [seller_id#1498]
            +- InMemoryRelation [order_id#1496, product_id#1497, seller_id#1498, date#1499, num_pieces_sold#1500, bill_raw_text#1501], StorageLevel(disk, memory, deserialized, 1 replicas)
                  +- *(1) ColumnarToRow
                     +- FileScan parquet [order_id#1496,product_id#1497,seller_id#1498,date#1499,num_pieces_sold#1500,bill_raw_text#1501] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/content/sales_parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<order_id:string,product_id:string,seller_id:string,date:string,num_pieces_sold:string,bill...


